## Librerias

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from word2number import w2n

In [ ]:
#pip install word2number

## Funcion

In [3]:
def scrape_books(min_rating, max_price):
    books_links = []
    for i in range(50):
        link = f"https://books.toscrape.com/catalogue/page-{i+1}.html"
        response = requests.get(link)
        response.status_code
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        book_headers = soup.find_all("h3")
        for element in book_headers:
            books_links.append(element.find("a").get("href"))
    upc_list = []
    title_list = []
    price_list = []
    rating_list = []
    genre_list = []
    availability_list = []
    description_list = []
    for element in books_links:
        link = "https://books.toscrape.com/catalogue/"+element
        response = requests.get(link)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        upc_list.append(soup.find_all("td")[0].get_text())
        title_list.append(soup.find("h1").get_text())
        price_list.append(soup.find(class_="price_color").get_text()[2:])
        rating_list.append(w2n.word_to_num(soup.find(class_="star-rating").get("class")[1]))
        genre_list.append(soup.find_all("li")[2].get_text(strip = True))
        availability_list.append(soup.find(class_="instock availability").get_text(strip=True))
        description_list.append(soup.find_all("p")[3].get_text(strip = True))
    dictionary = {"upc": upc_list,
                 "title": title_list,
                 "price (£)": price_list,
                 "rating": rating_list,
                 "genre": genre_list,
                 "availavility": availability_list,
                 "description": description_list}
    books_df = pd.DataFrame(data = dictionary)
    books_df["price (£)"] = books_df["price (£)"].astype(float)
    filtered_df = books_df[(books_df["rating"] >= min_rating) & (books_df["price (£)"] <= max_price)].reset_index(drop = True)
    return filtered_df

In [5]:
scrape_books(3.8,30)

,upc,title,price (£),rating,genre,availavility,description
0,e10e1e165dc8be4a,The Boys in the Boat: Nine Americans and Their...,22.60,4,Default,In stock (19 available),For readers of Laura Hillenbrand's Seabiscuit ...
1,30a7f60cd76ca58c,Shakespeare's Sonnets,20.66,4,Poetry,In stock (19 available),This book is an important and complete collect...
2,ce6396b0f23f6ecc,Set Me Free,17.46,5,Young Adult,In stock (19 available),Aaron Ledbetterâs future had been planned ou...
3,c2e46a2ee3b4a322,Chase Me (Paris Nights #2),25.27,5,Romance,In stock (19 available),"A Michelin two-star chef at twenty-eight, Viol..."
4,6258a1f6a6dcfe50,The Four Agreements: A Practical Guide to Pers...,17.66,5,Spirituality,In stock (18 available),"In The Four Agreements, don Miguel Ruiz reveal..."
...,...,...,...,...,...,...,...
140,bcbcbcf0f6ed196f,"Paper Girls, Vol. 1 (Paper Girls #1-5)",21.71,4,Sequential Art,In stock (1 available),"In the early hours after Halloween of 1988, fo..."
141,d510567580c8be52,Lost Among the Living,27.70,4,Historical Fiction,In stock (1 available),"England, 1921. Three years after her husband, ..."
142,a812f6969ddf3e39,Choosing Our Religion: The Spiritual Lives of ...,28.42,4,Religion,In stock (1 available),"To the dismay of religious leaders, study afte..."
143,19fec36a1dfb4c16,A Spy's Devotion (The Regency Spies of London #1),16.97,5,Historical Fiction,In stock (1 available),"In Englandâs Regency era, manners and elegan..."
